## Installation Instructions

Download and install miniconda: https://conda.io/miniconda.html

Make sure you are using the conda-forge channel:
```bash
$ conda config --add channels conda-forge
$ conda update --yes conda python
```

Install gsshapy:
```bash
$ conda create -n gssha python=2
$ source activate gssha
(gssha)$ conda install --yes gsshapy
```

In [ ]:
from datetime import datetime, timedelta
import os
try:
    from  urllib import urlretrieve
except ImportError:
    from urllib.request import urlretrieve

from gsshapy.modeling import GSSHAModel

Parameters to change for the run:

In [ ]:
base_dir = os.getcwd()
gssha_model_name = 'philippines_example'
land_use_grid_id = 'glcf'
gssha_model_directory = os.path.join(base_dir, gssha_model_name)
# make the directory for the output
try:
    os.mkdir(gssha_model_directory)
except OSError:
    pass

Download files:

In [ ]:
base_boundary_url = ('https://github.com/CI-WATER/gsshapy/'
                     'raw/master/tests/grid_standard/'
                     'phillipines/')
base_shape_filename = 'phillipines_5070115700'


# retrieve the shapefile
shapefile_name = base_shape_filename+'.shp'
boundary_shapefile = urlretrieve(base_boundary_url+shapefile_name,
                                 filename=os.path.join(gssha_model_directory, shapefile_name))[0]
for file_extension in ['.shx', '.prj', '.dbf']:
    file_name = base_shape_filename+file_extension
    urlretrieve(base_boundary_url+file_name, 
                filename=os.path.join(gssha_model_directory, file_name))
    
# retrieve the DEM
elevation_file_path = urlretrieve(base_boundary_url + 'gmted_elevation.tif', 
                                  filename=os.path.join(gssha_model_directory, 'gmted_elevation.tif'))[0]
# retrieve the land use grid
land_cover_url = ('https://github.com/CI-WATER/gsshapy/'
                  'raw/master/tests/grid_standard/'
                  'land_cover/LC_hd_global_2012.tif')
land_use_file_path = urlretrieve(land_cover_url, 
                                 filename=os.path.join(gssha_model_directory, 'LC_hd_global_2012.tif'))[0]

Use the data to generate a GSSHA model:

In [ ]:
# generate GSSHA model files
model = GSSHAModel(project_name=gssha_model_name,
                   project_directory=gssha_model_directory,
                   mask_shapefile=boundary_shapefile,
                   elevation_grid_path=elevation_file_path,
                   land_use_grid=land_use_file_path,
                   land_use_grid_id=land_use_grid_id,
                   out_hydrograph_write_frequency=1,
                   load_rasters_to_db=False)

# add card for max depth
model.project_manager.setCard('FLOOD_GRID',
                              '{0}.fgd'.format(gssha_model_name),
                              add_quotes=True)
# TODO: Add depth grids to simulation
# MAP_FREQ, DEPTH

# add event for simulation
model.set_event(simulation_start=datetime.utcnow(),
                simulation_duration=timedelta(seconds=2*60),
                rain_intensity=24,
                rain_duration=timedelta(seconds=1*60),
                )
model.write()

More information about options can be found here: http://gsshapy.readthedocs.io/en/latest/api/modeling/modeling.html